<a href="https://colab.research.google.com/github/PoppyDalamaZain/Kelompok3/blob/main/worksheet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Import Packages



In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings; warnings.simplefilter('ignore')
from datetime import datetime

#Memanggil Data

In [13]:
!mkdir data
!wget -P data/ https://raw.githubusercontent.com/PoppyDalamaZain/Kelompok3/main/data/Case-Study-02-MixITup_kelompok3.csv
df = pd.read_csv("data/Case-Study-02-MixITup_kelompok3.csv", low_memory = False, encoding='utf8')
    
df.shape

mkdir: cannot create directory ‘data’: File exists
--2022-10-12 01:48:58--  https://raw.githubusercontent.com/PoppyDalamaZain/Kelompok3/main/data/Case-Study-02-MixITup_kelompok3.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10688 (10K) [text/plain]
Saving to: ‘data/Case-Study-02-MixITup_kelompok3.csv.1’

Case-Study-02-MixIT 100%[===================>]  10.44K  --.-KB/s    in 0s      

2022-10-12 01:48:58 (119 MB/s) - ‘data/Case-Study-02-MixITup_kelompok3.csv.1’ saved [10688/10688]



(200, 10)

Melihat Varibel

In [14]:
df.columns

Index(['ID', 'Jenis Kelamin', 'Umur', 'Pendapatan-per-bulan',
       'skor-pengeluaran', 'Menikah', 'Kota', 'Promo', 'HariFav', 'FavFlavor'],
      dtype='object')

Jumlah Baris dan Kolom pada data

In [15]:
B, K = df.shape
print('Baris = ', B, ', Kolom = ', K)

Baris =  200 , Kolom =  10


Melihat Tipe dari setiap Variabel

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 200 entries, 0 to 199
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   ID                    200 non-null    int64 
 1   Jenis Kelamin         200 non-null    object
 2   Umur                  200 non-null    int64 
 3   Pendapatan-per-bulan  200 non-null    object
 4   skor-pengeluaran      200 non-null    int64 
 5   Menikah               200 non-null    int64 
 6   Kota                  200 non-null    object
 7   Promo                 200 non-null    int64 
 8   HariFav               200 non-null    object
 9   FavFlavor             200 non-null    int64 
dtypes: int64(6), object(4)
memory usage: 15.8+ KB


Mengubah object menjadi kategorik

In [19]:

df['Jenis Kelamin'] = df['Jenis Kelamin'].astype('category')
#df['Pendapatan-per-bulan'] = df['Pendapatan-per-bulan'].astype('int')
df['Menikah'] = df['Menikah'].astype('category')
df['Kota'] = df['Kota'].astype('category')
df['Promo'] = df['Promo'].astype('category')
df['HariFav'] = df['HariFav'].astype('category')
df['FavFlavor'] = df['FavFlavor'].astype('category')


Menghapus variabel yang tidak diperlukan

In [20]:
data_baru = df.drop(columns = ['ID'])
data_baru

,Jenis Kelamin,Umur,Pendapatan-per-bulan,skor-pengeluaran,Menikah,Kota,Promo,HariFav,FavFlavor
0,Wanita,49,Rp13.114.754,52,1,Jakarta,1,Sabtu,2
1,Wanita,32,Rp33.606.557,86,1,Depok,1,Minggu,6
2,Pria,51,Rp15.983.606,41,1,Jakarta,2,Minggu,3
3,Pria,19,Rp12.704.918,55,0,Jakarta,1,Rabu,7
4,Wanita,38,Rp20.081.967,42,1,Jakarta,2,Senin,8
...,...,...,...,...,...,...,...,...,...
195,Pria,31,Rp4.098.360,73,1,Depok,1,Senin,0
196,Pria,57,Rp15.983.606,51,1,Jakarta,1,Minggu,5
197,Wanita,35,Rp43.032.786,79,1,Depok,1,Senin,7
198,Wanita,23,Rp1.229.508,94,1,Depok,0,Selasa,0


In [21]:
data_baru.isnull().sum()

Jenis Kelamin           0
Umur                    0
Pendapatan-per-bulan    0
skor-pengeluaran        0
Menikah                 0
Kota                    0
Promo                   0
HariFav                 0
FavFlavor               0
dtype: int64

MENGISI MISSING VALUE DI EXCELL

INTERPRETASI :

Pengecekan Kembali

In [22]:
data_baru.duplicated().sum()

0

#Menentukan Clustering pada Data

In [ ]:
k=3
kmPP = cluster.KMeans(n_clusters=k, init='k-means++', max_iter=300, tol=0.0001, random_state = random_state)
kmPP.fit(X)
C_kmpp = kmPP.predict(X)

sns.countplot(x=C_kmpp)
C_kmpp[:10]